In [46]:
import os
import json
from collections import Counter
import pandas as pd 
import numpy as np
import cleanlab
from cleanlab.pruning import get_noise_indices

In [21]:
raw_df = pd.read_csv('results/cv0_prob_result.tsv', sep = '\t')
for i in range(1, 4):
    cv_df = pd.read_csv('results/cv{}_prob_result.tsv'.format(i), sep = '\t')
    raw_df = pd.concat([raw_df, cv_df], axis = 0)

In [22]:
raw_df.head()

,Integer_id,Image_id,Product_id,Prdtypecode,class_0_prob,class_1_prob,class_2_prob,class_3_prob,class_4_prob,class_5_prob,...,class_18_prob,class_19_prob,class_20_prob,class_21_prob,class_22_prob,class_23_prob,class_24_prob,class_25_prob,class_26_prob,class_pred
0,0,1263597046,3804725264,0,0.949759,0.014861,1.251165e-06,0.000004,3.224571e-03,0.003874,...,1.168206e-04,9.626128e-07,7.188042e-04,2.080143e-05,2.205838e-05,2.375256e-03,2.981653e-06,1.061014e-03,7.087391e-05,0.0
1,1,1008141237,436067568,1,0.000016,0.999315,6.952421e-08,0.000002,3.168643e-06,0.000005,...,5.844390e-04,7.533432e-07,7.498671e-08,2.037524e-07,5.220247e-07,1.482183e-06,6.881482e-08,2.065684e-07,8.151939e-07,1.0
2,2,938777978,201115110,2,0.000025,0.000002,2.444098e-05,0.000831,6.427478e-08,0.918070,...,1.301149e-05,8.052827e-08,8.395377e-07,5.486393e-02,3.362804e-06,2.248354e-07,8.309863e-03,2.122613e-05,1.005403e-05,5.0
3,3,457047496,50418756,3,0.000003,0.000009,1.043968e-06,0.004990,9.777119e-05,0.000004,...,9.568751e-08,9.603481e-10,1.527118e-06,3.174247e-07,4.565350e-08,1.871020e-04,2.456394e-09,1.384700e-07,1.299823e-08,11.0
4,4,1077757786,278535884,4,0.020966,0.000020,7.478552e-08,0.000005,9.746689e-01,0.000002,...,5.270756e-05,3.975110e-08,4.708430e-08,5.438192e-09,6.632543e-09,3.393239e-07,2.572840e-09,1.647421e-07,1.227326e-08,4.0


In [24]:
raw_df = raw_df.reset_index(drop = True)

In [7]:
# get noisy label array
noisy_labels = []
for i in range(4):
    cv_df = pd.read_csv('results/cv{}_prob_result.tsv'.format(i), sep = '\t')
    noisy_labels.extend(cv_df['Prdtypecode'].values.tolist())

In [8]:
len(noisy_labels)

84916

In [10]:
# read psx numpy array
psx = np.load('results/psx.npy')

In [11]:
psx.shape

(84916, 27)

In [13]:
s = noisy_labels

### Method 1

In [16]:
ordered_label_errors = get_noise_indices(
    s=noisy_labels,
    psx=psx,
    sorted_index_method='normalized_margin', # Orders label errors
 )

In [28]:
len(ordered_label_errors)

21200

In [20]:
ordered_label_errors[:10]

array([74881, 82528, 16736,  2966, 28715, 23556,  4206, 16604,  1810,
       14739])

In [44]:
label_error_df = raw_df.loc[ordered_label_errors]

In [45]:
label_error_df.head()

,Integer_id,Image_id,Product_id,Prdtypecode,class_0_prob,class_1_prob,class_2_prob,class_3_prob,class_4_prob,class_5_prob,...,class_18_prob,class_19_prob,class_20_prob,class_21_prob,class_22_prob,class_23_prob,class_24_prob,class_25_prob,class_26_prob,class_pred
74881,74881,1264049685,3923104289,20,4.216706e-19,1.314919e-18,9.561437e-19,9.028184e-12,5.846277e-21,2.631345e-15,...,8.109685e-16,8.315966e-19,5.138054e-12,4.279560e-14,1.097847e-12,2.727769e-20,9.961632e-14,3.388623e-17,1.000000e+00,26.0
82528,82528,1186390521,3028261446,3,3.696276e-14,1.461823e-15,8.769931e-16,8.705419e-12,6.752231e-17,3.938758e-14,...,4.942682e-15,2.159873e-16,1.000000e+00,9.594851e-15,4.847140e-10,1.940777e-18,9.421690e-13,2.771635e-12,2.833727e-13,20.0
16736,16736,1282607000,4057738505,17,2.369414e-18,1.017903e-18,1.075509e-18,1.149331e-13,2.415732e-18,2.279298e-16,...,7.136349e-17,6.538634e-21,3.506180e-17,7.077884e-13,5.032988e-20,6.636671e-19,1.000000e+00,1.067617e-17,1.260719e-14,24.0
2966,2966,1144092318,2288640592,20,2.332115e-10,6.370761e-15,1.003441e-12,2.550831e-10,1.407191e-13,1.811568e-11,...,2.261168e-12,7.967960e-14,7.512955e-10,6.831529e-12,2.534059e-09,9.949669e-14,7.621456e-12,1.000000e+00,1.295210e-11,25.0
28715,28715,1317472729,4213278726,6,4.002297e-13,4.070668e-14,9.628805e-11,1.257896e-11,5.906725e-17,3.323368e-12,...,7.877012e-13,3.235475e-14,2.730200e-09,3.093407e-14,3.298622e-14,3.680254e-16,1.268015e-12,1.844490e-13,5.348747e-12,7.0


In [49]:
Counter(label_error_df['Prdtypecode'].values[:2120]).most_common(27)

[(3, 278),
 (7, 203),
 (17, 182),
 (6, 162),
 (20, 135),
 (8, 116),
 (10, 114),
 (5, 87),
 (24, 84),
 (0, 83),
 (12, 70),
 (2, 69),
 (21, 69),
 (18, 60),
 (9, 58),
 (1, 57),
 (14, 51),
 (15, 50),
 (13, 45),
 (11, 38),
 (22, 36),
 (26, 19),
 (4, 15),
 (23, 13),
 (19, 11),
 (25, 10),
 (16, 5)]

In [76]:
train = pd.read_csv('/home/competition/SIGIR2020/Task_1/data/preprocessed/train_0_clean.tsv', sep = '\t')
valid = pd.read_csv('/home/competition/SIGIR2020/Task_1/data/preprocessed/valid_0_clean.tsv', sep = '\t')

In [51]:
valid.head()

,Integer_id,Title,Description,Image_id,Product_id,Prdtypecode
0,11,Fauteuil Chesterfield Brenton 100% Cuir De Buf...,Canapé et fauteuil chesterfield en cuir antiqu...,1295816984,4127967621,7
1,12,Peaceable Kingdom Wheres Bear? The Hide And Fi...,pCan my 2-year-old play a game? Yes Each game ...,1204199842,3287127001,8
2,26,Modèle De Voiture 4pcs Alliage Métallique 1.9i...,4pcs alliage métallique 1.9in jante de roue po...,1297749086,4137059841,3
3,33,Lindner 2365-2115ce Coin Case Nera Xl With 3 T...,Timeless elegance a clear design versatility f...,1248853903,3817904723,5
4,56,Ne Regardez Pas Le Voleur Qui Passe,Depuis quatre ans le mensuel Marie Claire orga...,1071868057,476383013,4


In [68]:
valid[valid['Integer_id'].isin(label_error_df['Integer_id'].values[:2120])]

,Integer_id,Title,Description,Image_id,Product_id,Prdtypecode
8,73,Pot Marina Eda - Diamètre 25 Cm - 6 L - Rubis,Pot Marina Eda - Diamètre 25 cm - 6 l - Rubis...,1166636908,2823144007,6
39,369,Pochoir Plastique Animaux - Paquet De 6,NaN,862537855,102358907,5
69,677,Cravate Délicate Pour Rideau D'ours Décoration...,La description: La tenue de rideau exquise à ...,1319272516,4214120496,17
81,800,Ensemble De Mobilier De Jardin - 4 Places - 1 ...,Ensemble de mobilier de jardin - Dimensions ta...,1307347398,4171370647,6
162,1543,Lampe Led Humidificateur Coupe Lumière Mignon ...,Lampe Humidifier Mignon Coupe Lumière LED Humi...,1252097056,3847027140,17
...,...,...,...,...,...,...
8283,82786,Tobi Matratze Babybay Original Medicott Extral...,Tobi Matratze babybay original medicott extral...,1078180838,1395653000,20
8327,83375,Raccord En T 1/4 Pour Brumisateur Haute Perfor...,Raccord en T 1/4,1268772457,3924558661,6
8345,83559,Identity In Latin American And Latina Literatu...,NaN,1038104558,427668486,0
8451,84514,Crème Glacée 1pc Enfants Bols Tasse De Crème G...,1 x bol de crème glacée avec une cuillère Cara...,1256911490,3875020691,17


In [77]:
train[train['Integer_id'].isin(label_error_df['Integer_id'].values[:2120])]

,Integer_id,Title,Description,Image_id,Product_id,Prdtypecode
3,3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,457047496,50418756,3
15,17,Mod Podge Dishwasher Safe Gloss 8oz,NaN,1156191369,2666371748,5
83,95,Magasin Zhufuwme:English Alphabet Coussin Impr...,English Alphabet Coussin Imprimer Coussin Fleu...,1280265865,4044463867,7
119,135,Dinosaur Bubble Light Up Bulles Ventilat Avec ...,Dinosaur Bubble Light Up Bulles ventilateur av...,1275716747,4006540302,8
146,168,Figurine Porcelaine Chat Blanc Fleur Papier,NaN,1132633996,2230071933,17
...,...,...,...,...,...,...
76262,84739,8 Chapeaux Pointus Joyeux Anniversaire K,Lot de 8 chapeaux pointus avec impression Joye...,1310680337,4128368070,17
76303,84784,Modelcraft Adaptateur Connecteur Du Câble Jst ...,Adaptateur connecteur JST câble pour fiche T (...,1005833496,418197018,14
76318,84800,Tapis de sol en mousse 177 cm² - MSpa,Mousse de protection sol 177 cm² - MSpa à prix...,1246418213,3798934097,12
76390,84878,Lisette N° 4 Et 5,NaN,1235516479,3690846059,1


In [78]:
train_new = train[~train['Integer_id'].isin(label_error_df['Integer_id'].values[:2120])]
train_new.to_csv('/home/competition/SIGIR2020/Task_1/data/preprocessed/train_0_clean_noise_reduce_2120.tsv', index = False, sep = '\t')

In [70]:
valid_new = valid[~valid['Integer_id'].isin(label_error_df['Integer_id'].values[:2120])]
valid_new.to_csv('/home/competition/SIGIR2020/Task_1/data/preprocessed/valid_0_clean_noise_reduce_2120.tsv', index = False, sep = '\t')

In [79]:
train_new.shape, valid_new.shape

((74490, 6), (8306, 6))

In [69]:
np.save('label_error_integer_ids_2120.npy', label_error_df['Integer_id'].values[:2120])

In [72]:
cls_cnt_dict = dict(Counter(valid['Prdtypecode'].values))
cls_cnt_dict_new = dict(Counter(valid_new['Prdtypecode'].values))

In [73]:
for cls in range(27):
    print(cls_cnt_dict[cls], cls_cnt_dict_new[cls])

312 303
476 474
168 160
487 462
276 275
499 487
259 242
507 495
207 193
430 424
478 471
267 264
1021 1011
76 70
505 504
142 135
395 394
499 483
251 248
83 82
324 314
249 246
83 81
87 86
250 245
80 78
81 79


In [ ]:
label_error_mask = np.zeros(len(s), dtype=bool)
label_error_indices = compute_confident_joint(
    s, psx, return_indices_of_off_diagonals=True
)[1]
for idx in label_error_indices:
    label_error_mask[idx] = True
baseline_conf_joint_only = label_error_mask

In [ ]:
# Method: C_confusion
baseline_argmax = baseline_methods.baseline_argmax(psx, s)

In [ ]:
# Method: CL: PBC
baseline_cl_pbc = cleanlab.pruning.get_noise_indices(
            s, psx, prune_method='prune_by_class')

In [ ]:
# Method: CL: PBNR
baseline_cl_pbnr = cleanlab.pruning.get_noise_indices(
            s, psx, prune_method='prune_by_noise_rate')

In [37]:
# Method: CL: C+NR
baseline_cl_both = cleanlab.pruning.get_noise_indices(
            s, psx, prune_method='both')

In [40]:
raw_df[baseline_cl_both]

,Integer_id,Image_id,Product_id,Prdtypecode,class_0_prob,class_1_prob,class_2_prob,class_3_prob,class_4_prob,class_5_prob,...,class_18_prob,class_19_prob,class_20_prob,class_21_prob,class_22_prob,class_23_prob,class_24_prob,class_25_prob,class_26_prob,class_pred
2,2,938777978,201115110,2,2.490332e-05,1.892824e-06,2.444098e-05,8.313152e-04,6.427478e-08,9.180701e-01,...,1.301149e-05,8.052827e-08,8.395377e-07,5.486393e-02,3.362804e-06,2.248354e-07,8.309863e-03,2.122613e-05,1.005403e-05,5.0
3,3,457047496,50418756,3,2.748470e-06,8.682372e-06,1.043968e-06,4.990175e-03,9.777119e-05,3.659020e-06,...,9.568751e-08,9.603481e-10,1.527118e-06,3.174247e-07,4.565350e-08,1.871020e-04,2.456394e-09,1.384700e-07,1.299823e-08,11.0
5,5,393356830,5862738,1,1.622404e-01,1.335496e-03,1.256644e-09,2.133688e-09,8.005591e-02,1.331208e-07,...,9.975135e-08,7.066930e-10,5.321928e-10,7.744206e-09,9.250071e-10,3.009000e-09,3.764217e-09,2.476614e-09,6.949476e-10,10.0
6,6,907794536,91920807,0,9.766605e-02,4.784310e-06,1.158892e-06,5.719343e-07,9.009572e-01,2.167695e-04,...,6.739194e-07,1.293416e-07,9.906280e-08,4.934220e-09,3.484011e-08,4.937953e-07,2.275346e-08,2.619571e-06,2.197912e-06,4.0
17,17,1156191369,2666371748,5,1.224596e-08,5.828246e-10,8.428967e-09,1.649732e-07,1.216324e-09,3.386510e-08,...,2.100616e-08,2.953206e-10,6.008042e-07,1.473720e-08,8.367322e-07,2.158124e-10,1.838414e-06,4.874782e-07,3.282425e-08,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84901,84901,1006234772,421484600,11,2.813720e-11,1.454852e-11,2.491133e-05,2.105355e-04,6.883875e-11,3.639547e-09,...,1.484858e-06,1.578379e-07,1.198181e-08,1.240321e-06,1.668268e-09,1.455859e-07,1.603111e-11,7.776313e-11,2.862279e-11,15.0
84903,84903,1275489688,4006488781,8,4.879018e-07,1.450447e-08,1.311464e-05,2.267913e-02,1.821442e-07,2.703889e-05,...,9.057171e-06,6.395740e-07,9.145465e-05,9.629718e-01,4.947338e-05,1.777696e-06,2.810778e-04,8.242048e-06,2.570981e-05,21.0
84904,84904,1309066528,4179411158,7,1.391420e-06,4.361875e-07,1.257567e-04,4.198659e-06,9.229828e-07,4.077121e-02,...,1.340254e-03,5.156270e-06,1.512026e-03,4.836479e-04,6.028055e-03,3.867687e-08,1.329594e-02,2.268149e-03,1.064431e-03,17.0
84906,84906,1089840761,1551455565,11,5.044590e-05,1.056746e-05,1.316839e-04,7.893189e-02,1.858169e-06,8.731686e-04,...,4.001849e-05,7.779053e-06,2.315226e-03,1.331018e-03,5.381701e-04,2.867964e-06,1.437385e-02,4.765488e-05,8.321459e-04,14.0
